## Librerias

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

## Variables

In [0]:
def variables_globales():
    return {
        "container": dbutils.secrets.get("scope-mbc", "secret-env-container"),
        "storage_account": dbutils.secrets.get("scope-mbc", "secret-env-storage-account"),
        "path_base": f"abfss://{dbutils.secrets.get('scope-mbc', 'secret-env-container')}@{dbutils.secrets.get('scope-mbc', 'secret-env-storage-account')}.dfs.core.windows.net"
    }

In [0]:
print(variables_globales()["path_base"])

## Funciones

In [0]:
def read_landing(spath: str):
    path_base = variables_globales()["path_base"]
    df = spark.read.format("parquet").load(f"{path_base}/{path}")

    return df_final